In [1]:
!pip install -qq adapters datasets

In [2]:
!pip uninstall -y transformers adapters peft trl accelerate tokenizers
!pip install transformers==4.51.3
!pip install adapters==1.2.0
!pip install accelerate==0.26.0

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: adapters 1.2.0
Uninstalling adapters-1.2.0:
  Successfully uninstalled adapters-1.2.0
Found existing installation: accelerate 0.26.0
Uninstalling accelerate-0.26.0:
  Successfully uninstalled accelerate-0.26.0
Found existing installation: tokenizers 0.21.4
Uninstalling tokenizers-0.21.4:
  Successfully uninstalled tokenizers-0.21.4


  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl (2.5 MB)

   ---------------------------------------- 0/2 [tokenizers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ----------------

In [3]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict, load_dataset
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizer, RobertaConfig, TrainingArguments, EvalPrediction
from adapters import AutoAdapterModel, AdapterTrainer

c:\Users\apala\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data_tom = load_dataset("rotten_tomatoes")
data_tom.num_rows

{'train': 8530, 'validation': 1066, 'test': 1066}

In [5]:
print(data_tom)
print(data_tom["train"][2])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})
{'text': 'effective but too-tepid biopic', 'label': 1}


In [6]:
cora = pd.read_csv("../Hyperedges/Cora_dataset.csv")

le = LabelEncoder()
cora['labels'] = le.fit_transform(cora['topic2']) 

cora_ft = Dataset.from_pandas(cora[["Abstract", "labels"]])
cora_ft = cora_ft.rename_column("Abstract", "text")

In [7]:
cora_ft = cora_ft.train_test_split(test_size=0.2, seed=42)
test_valid = cora_ft["test"].train_test_split(test_size=0.5, seed=42)

dataset_dict = {
    "train": cora_ft["train"],
    "validation": test_valid["train"],
    "test": test_valid["test"]
}

dataset = DatasetDict(dataset_dict)

In [8]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
    return tokenizer(
        batch["text"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

dataset = dataset.map(encode_batch, batched=True)
dataset = dataset.with_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map: 100%|██████████| 237/237 [00:00<00:00, 637.97 examples/s]


In [9]:
config = RobertaConfig.from_pretrained("roberta-base",num_labels=7)
model = AutoAdapterModel.from_pretrained("roberta-base", config=config)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
n_labels = len(set(cora["topic2"]))
id2label = {i: lab for i, lab in enumerate(le.classes_)}
# Add a new adapter
model.add_adapter("cora_fn", config="seq_bn")

# Add a matching classification head
model.add_classification_head(
    "cora_fn",
    num_labels=n_labels,
    id2label=id2label,
  )

# Activate the adapter
model.train_adapter("cora_fn")

There are adapters available but none are activated for the forward pass.


In [11]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=9,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    logging_steps=50,
    output_dir="./cora_fn_out",
    overwrite_output_dir=True,
    report_to=[],
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"accuracy": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [12]:
trainer.train()

Step,Training Loss
50,1.882500
100,1.843200
150,1.356100
200,0.978700
250,0.936200
300,0.744800
350,0.761400
400,0.689700
450,0.643700
500,0.607400


TrainOutput(global_step=1071, training_loss=0.7617467208848058, metrics={'train_runtime': 7229.0951, 'train_samples_per_second': 2.358, 'train_steps_per_second': 0.148, 'total_flos': 1149150069080064.0, 'train_loss': 0.7617467208848058, 'epoch': 9.0})

In [13]:
trainer.evaluate()

{'eval_loss': 0.5739740133285522,
 'eval_accuracy': 0.8185654008438819,
 'eval_runtime': 23.8145,
 'eval_samples_per_second': 9.952,
 'eval_steps_per_second': 0.336,
 'epoch': 9.0}